In [27]:
import pandas as pd
import numpy as np
import re
import os
import time
import translitua

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## Raw data

In [28]:
# Loading raw data
df = pd.read_csv('ria_raw.csv', encoding = 'windows-1251')

print(df.shape) # Rows, cols
df.head()

(261855, 4)


,parse,parse2,price_usd,parse3
0,/ru/realty-perevireno-prodaja-kvartira-kiev-so...,р-н. Соломенский ул. Ереванская 1 ком. г. Киев,71 970 $,1 комната 57 м кв
1,/ru/realty-perevireno-prodaja-kvartira-kiev-so...,р-н. Соломенский Машинобудівна вулиця 1 ком. г...,68 000 $,1 комната 68 м кв
2,/ru/realty-perevireno-prodaja-dom-vinnitsa-yak...,р-н. Якушинцы ул. Богдана Хмельницкого 134м кв...,53 000 $,134 м кв 7 соток
3,/ru/realty-perevireno-prodaja-kvartira-vinnits...,р-н. Свердловский массив ул. Матроса Кошки 1 к...,36 500 $,1 комната 36 м кв
4,/ru/realty-perevireno-prodaja-kvartira-vinnits...,р-н. Стрижавка Алеи улица 1 ком. г. Винница,16 582 $,1 комната 40.24 м кв


In [29]:
# Null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261855 entries, 0 to 261854
Data columns (total 4 columns):
parse        261855 non-null object
parse2       261855 non-null object
price_usd    260053 non-null object
parse3       247766 non-null object
dtypes: object(4)
memory usage: 8.0+ MB


### Removing null values 

In [30]:
df.dropna(subset = ['price_usd'], inplace = True)
print(df.shape)
df.head()

(260053, 4)


,parse,parse2,price_usd,parse3
0,/ru/realty-perevireno-prodaja-kvartira-kiev-so...,р-н. Соломенский ул. Ереванская 1 ком. г. Киев,71 970 $,1 комната 57 м кв
1,/ru/realty-perevireno-prodaja-kvartira-kiev-so...,р-н. Соломенский Машинобудівна вулиця 1 ком. г...,68 000 $,1 комната 68 м кв
2,/ru/realty-perevireno-prodaja-dom-vinnitsa-yak...,р-н. Якушинцы ул. Богдана Хмельницкого 134м кв...,53 000 $,134 м кв 7 соток
3,/ru/realty-perevireno-prodaja-kvartira-vinnits...,р-н. Свердловский массив ул. Матроса Кошки 1 к...,36 500 $,1 комната 36 м кв
4,/ru/realty-perevireno-prodaja-kvartira-vinnits...,р-н. Стрижавка Алеи улица 1 ком. г. Винница,16 582 $,1 комната 40.24 м кв


### "Parse" column preprocessing 

In [31]:
# Extracting offer type
df['parse'] = df['parse'].apply(lambda x: x.replace('_', '-'))
df['parse'] = df['parse'].apply(lambda x: x.replace('perevireno-', '').replace('/ru/realty-', ''))
df['parse'] = df['parse'].apply(lambda x: x.replace('prodaja', 'prodaj').replace('prodaj', 'sell/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('dolgosrochnaya-arenda', 'lt_rent/').replace('dovgostrokova-orenda', 'lt_rent/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('arenda-posutochnaya', 'st_rent/').replace('orenda-podobova', 'st_rent/'))

df['offer_type'] = df['parse'].apply(lambda x: x.split('/')[0])
df['parse'] = df['parse'].apply(lambda x: x.split('/-')[1])

# Extracting type of real estate
df['parse'] = df['parse'].apply(lambda x: x.replace('kvartira-', 'apartments/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('dom-', 'house/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('budinok-', 'house/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('pomescheniya-svobodnogo-naznacheniya-', 'PSN/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('ofisnyie-pomescheniya-', 'office/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('ofisnye-pomescheniya-', 'office/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('ofisnoe-pomeschenie-', 'office/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('skladskie-pomescheniya-', 'warehouse/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('proizvodstvennye-pomescheniya-', 'industrial/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('proizvodstvennyie-pomescheniya-', 'industrial/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('uchastok-pod-jiluyu-zastroyku-', 'land/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('chast-doma-', 'part_of_house/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('zemlya-kommercheskogo-naznacheniya-', 'land/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('zemlya-selskohozyaystvennogo-naznacheniya-', 'land/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('komnata-', 'room/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('torgovyie-ploschadi-', 'commercial_area/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('torgovye-ploschadi-', 'commercial_area/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('obyekt-sferyi-uslug-', 'services/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('obyekt-sfery-uslug-', 'services/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('dacha', 'country_house/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('boks-v-garajnom-komplekse-', 'garage/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('otdelno-stoyaschiy-garaj-', 'garage/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('mesto-v-garajnom-kooperative-', 'parking_space/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('podzemnyiy-parking-', 'parking_space/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('podzemnyy-parking-', 'parking_space/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('mesto-na-stoyanke-', 'parking_space/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('ofisnyie-zdaniya-', 'office_complex/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('ofisnye-zdaniya-', 'office_complex/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('ofisnoe-zdanie-', 'office_complex/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('otel-gostinitsa-', 'hotel/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('kafe-bar-restoran-', 'recreational/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('baza-otdyha-pansionat-', 'recreational/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('baza-otdyiha-pansionat-', 'recreational/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('gotovyy-biznes-', 'business/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('gotovyiy-biznes-', 'business/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('zemlya-rekreatsionnogo-naznacheniya-', 'land/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('zemlya-prirodnozapovednogo-naznacheniya-', 'land/'))
df['parse'] = df['parse'].apply(lambda x: x.replace('zemlya-prirodno-zapovednogo-naznacheniya-', 'land/'))

df['type'] = df['parse'].apply(lambda x: x.split('/')[0])

# Drop 'parse' column
df.drop('parse', axis = 1, inplace = True)

In [32]:
# Data after "parse" column preprocessing
df.head()

,parse2,price_usd,parse3,offer_type,type
0,р-н. Соломенский ул. Ереванская 1 ком. г. Киев,71 970 $,1 комната 57 м кв,sell,apartments
1,р-н. Соломенский Машинобудівна вулиця 1 ком. г...,68 000 $,1 комната 68 м кв,sell,apartments
2,р-н. Якушинцы ул. Богдана Хмельницкого 134м кв...,53 000 $,134 м кв 7 соток,sell,house
3,р-н. Свердловский массив ул. Матроса Кошки 1 к...,36 500 $,1 комната 36 м кв,sell,apartments
4,р-н. Стрижавка Алеи улица 1 ком. г. Винница,16 582 $,1 комната 40.24 м кв,sell,apartments


### "Parse2" column preprocessing

In [34]:
# Extracting city names
df['city'] = df['parse2'].apply(lambda x: re.findall(r'г\.\s\w+', x)[-1].replace('г. ', ''))

cities = {
'Івано': 'Ивано Франковск', 'Голая': 'Голая Пристань', 'Каменец': 'Каменец-Подольский', 'Новая': 'Новая Каховка',
'Berezan': 'Березань', 'Burshtyn': 'Бурштын',
'Gnivan': 'Гнивань', 'Lubotun': 'Люботин', 'Noviy': 'Новый роздол', 'Uzhgorod': 'Ужгород', 'rzishev': 'Ржев', 'Іршавa': 'Иршава',
'Белая': 'Белая Церковь', 'Бердичів': 'Бердичев', 'Богородчани': 'Богородчани', 'Бориспіль': 'Борисполь', 'Борщів': 'Борщев', 'Буськ': 'Буск', 'Бучач': 'Буча',
'Гола': 'Голая Пристань', 'Залізний': 'Железный порт', 'Киево': 'Киево-Святошинский', 'Кривой': 'Кривой рог', 
'Малая': 'Малая Виска', 'Нова': 'Нова Ушиця', 'Новий': 'Новий Роздол', 'Новые': 'Новые Санжары', 'Новый': 'Новый Буг',
'Овідіополь': 'Овидиополь', 'Одеса': 'Одесса', 'Оржиця': 'Оржица', 'П': 'Пятихатки', 'Старая': 'Старая Синява', 'Старий': 'Старый Самбор', 'Старый': 'Старый Самбор', 
'Ивано': 'Івано-Франківськ', 'Ильичевск': 'Чорноморськ', 'Литин': 'Літин', 'Соленое': 'Солоне', 'Межгорье': "Міжгір'я",
'Александрия': 'Олександрія', 'Чертков': 'Чортків', 'Раздельная': 'Роздільна', 'Красный': 'Хрустальный', 'Белгород': 'Білгород-Дністровський',
'Гребенка': 'Гребінка', 'Бобровица': 'Бобровиця', 'Первомайск': 'Первомайськ', 'Котовск': 'Подільськ', 'Орехов': 'Оріхів',
'Комсомольск': 'Горішні Плавні', 'Зборов': 'Зборів', 'Приморск': 'Приморськ', 'Великая': 'Велика Новосілка',
'Шацк': 'Шацьк', 'Васильковка': 'Васильківка', 'Семеновка': 'Семенівка', 'Ширяево': 'Ширяєве', 'Чутово': 'Чутове',
'Александровка': 'Олександрівка', 'Петрово': 'Петрове', 'Середина': 'Середина-Буда', 'Новгород': 'Новгород-Сіверський',
'Красноармейск': 'Покровськ', 'Дзержинск': 'Торецьк', 'Снигиревка': 'Снігурівка', 'Кривое': 'Кривий Ріг', 'Карловка': 'Карлівка',
'Куйбышево': 'Більмак', 'Михайловка': 'Михайлівка',    
}

df['city'] = df['city'].replace(cities)

# Drop 'parse2' column
df.drop('parse2', axis = 1, inplace = True)

### Region column creation 

In [36]:
# Nominatim instance for geocoding
geo = Nominatim(user_agent = 'thesis_donnu', timeout = 3)

# Dict to store region address
regions = dict()

for city in df['city'].unique():
    try:
        location = geo.geocode(city)
        print(location.address)
        regions[city] = location.address
    except:
        print(city, ' ERROR')
    time.sleep(1)

Київ, Шевченківський район, Київ, 1001, Україна
Вінниця, Вінницька область, 21000-21499, Україна
Одеса, Приморський район, Одеса, Одеська область, 65026, Україна
Львів, Галицький район, Львівська міська рада, Львівська область, Україна
Дніпро, Шевченківський район, Дніпровська міська рада, Дніпропетровська область, Україна
Ужгород, Закарпатська область, 88000-88499, Україна
Буча, Київська область, 08293, Україна
Харків, Шевченківський район, Харківська міська рада, Харківська область, Україна
Києво-Святошинський район, Київська область, Україна
Миколаїв, Центральний район, Миколаївська міська рада, Миколаївська область, Україна
Тернопіль, Тернопільська область, 46000-46499, Україна
Вишгород, Вишгородський район, Київська область, 07301, Україна
Хмельницький, Хмельницька область, 29000-29499, Україна
Запоріжжя, Вознесенівський район, Запорізька міська рада, Запорізька область, 69000-69499, Україна
Івано-Франківськ, Івано-Франківська міська рада, Івано-Франківська область, Україна
Черкас

Чортків, Тернопільська область, 48500-48509, Україна
Роздільна, Роздільнянський район, Одеська область, Україна
Дубровиця, Дубровицький район, Рівненська область, Україна
Черкаси, Соснівський район, Черкаська міська рада, Черкаська область, 18000-18499, Україна
Могилев, Боровской сельсовет, Болотнинский район, Новосибирская область, Сибирский федеральный округ, Россия
Ковель, Ковельська міська рада, Волинська область, Україна
Хрустальний, Краснолуцька міська рада, Луганська область, Україна
Баришівка, Баришівський район, Київська область, 07500-07571, Україна
Міжгір  ERROR
Борщів, Борщівська міська рада, Борщівський район, Тернопільська область, 48700, Україна
Калуш, Калуська міська рада, Івано-Франківська область, Україна
Млынов  ERROR
Теребовля, Теребовлянська міська рада, Теребовлянський район, Тернопільська область, 48100, Україна
Надвірна, Надвірнянський район, Івано-Франківська область, Україна
Шпола, Шполянський район, Черкаська область, Україна
Нова Каховка, Новокаховська міськ

Тростянець, Тростянецька міська об’єднана територіальна громада, Тростянецький район, Сумська область, 42600-42615, Україна
Дубно, Рівненська область, 35600-35608, Україна
Троїцьке, Троїцький район, Луганська область, 92102, Україна
Ставище, Ставищенський район, Київська область, 09400, Україна
Коломия, Івано-Франківська область, 78200, Україна
Березанка, Березанський район, Миколаївська область, 57400-57409, Україна
Баштанка, Баштанський район, Миколаївська область, 56100-56109, Україна
Галич, Галицький район, Івано-Франківська область, 77100, Україна
Березне, Березнівський район, Рівненська область, Україна
Білозерка, Білозерський район, Херсонська область, Україна
Конотоп, Конотопська міська об’єднана територіальна громада, Конотопська міська рада, Сумська область, 41600-41615, Україна
Каланчак, Каланчацький район, Херсонська область, Україна
Каменка, Каменский район, Пензенская область, Приволжский федеральный округ, Россия
Рокитне, Рокитнянський район, Київська область, 09600, Укр

Поліське, Рагівка, Поліський район, Київська область, 07033, Україна
Шепетівка, Шепетівська міська рада, Хмельницька область, Україна
Широке, Широківський район, Дніпропетровська область, 53700-53704, Україна
Первомайський, Первомайська міська рада, Харківська область, 64100-64109, Україна
Вижниця, Вижницький район, Чернівецька область, 59200, Україна
Краснополье, Краснопольский район, Могилёвская область, Беларусь
Піщанка, Піщанський район, Вінницька область, 24700, Україна
Врадіївка, Врадіївський район, Миколаївська область, 56301, Україна
Куликівка, Куликівський район, Чернігівська область, 16300, Україна
Новоукраїнка, Новоукраїнський район, Кіровоградська область, 27100-27108, Україна
Липовець, Липовецька міська рада, Липовецький район, Вінницька область, 22500-22505, Україна
Охтирка, Охтирська міська об'єднана територіальна громада, Охтирська міська рада, Сумська область, Україна
Лисянка, Лисянська селищна громада, Лисянський район, Черкаська область, 19300, Україна
Баранівка, Бар

Демидівка, Демидівський район, Рівненська область, 35200, Україна
Слов'яносербськ, Т-13-15, Лотикове, Слов’яносербський район, Луганська область, 93747, Україна
Володимирець, Володимирецький район, Рівненська область, 34300, Україна
Екатеринбург, городской округ Екатеринбург, Свердловская область, Уральский федеральный округ, Россия
Стара Синява, Старосинявський район, Хмельницька область, 31400, Україна
Старобешеве, Старобешівська селищна рада, Старобешівський район, Донецька область, 87200-87205, Україна
Вараш, Рівненська область, 34400, Україна
Тернівка, Тернівська міська рада, Дніпропетровська область, 51500-51509, Україна
Єланець, Єланецький район, Миколаївська область, 55507, Україна
Бровари, Київська область, 07400, Україна
Ратне, Ратнівський район, Волинська область, 44100, Україна
Глубокая, Анадырский район, Чукотский автономный округ, Дальневосточный федеральный округ, Россия
Шумське, Москалівка, Роменський район, Сумська область, 42062, Україна
Любар, Любарський район, Житом

In [37]:
clean_reg = dict() # Dict to store region name in ukrainian language
clean_city = dict() # Dict to store city name in ukrainian language

for city in regions.keys():
    # Kyiv considered as a separate region
    if city == 'Киев':
        clean_reg[city] = 'Київ'
        clean_city[city] = 'Київ'    
    
    elif 'Україна' in regions[city]:
        temp = regions[city].split(',')
        c = temp[0]
        temp = [i.strip() for i in temp if 'область' in i]
        if temp:
            region = temp[0]
            clean_reg[city] = region
            clean_city[city] = c
            
df['region'] = df['city']
df['region'] = df['region'].replace(clean_reg)

df.drop(df.loc[df['city'] == df['region'], 'city'].index, inplace = True)
df['city'] = df['city'].replace(clean_city)

In [38]:
# Transliteration
df['city'] = df['city'].apply(lambda x: translitua.translit(x))
df['region'] = df['region'].apply(lambda x: translitua.translit(x))
df['region'] = df['region'].apply(lambda x: x.split()[0])

In [39]:
df.head()

,price_usd,parse3,offer_type,type,city,region
0,71 970 $,1 комната 57 м кв,sell,apartments,Kyiv,Kyiv
1,68 000 $,1 комната 68 м кв,sell,apartments,Kyiv,Kyiv
2,53 000 $,134 м кв 7 соток,sell,house,Vinnytsia,Vinnytska
3,36 500 $,1 комната 36 м кв,sell,apartments,Vinnytsia,Vinnytska
4,16 582 $,1 комната 40.24 м кв,sell,apartments,Vinnytsia,Vinnytska


### "parse3" column preprocessing 

In [48]:
# Extracting number of rooms, living area and area of land
def rooms(x, mode):
    if mode == 'rooms':
        t = re.search(r'\d+\s\bком', x)
    elif mode == 'area':
        t = re.search(r'\d+\s\bм\s\bкв', x)
        if not t:
            t = re.search(r'\d+\s\bкв\.\s\bм', x)
    elif mode == 'land_area':
        t = re.search(r'\d+\s\bсот', x)
        
    if t:
        return re.findall(r'\d+', t[0])[0]
    else:
        return np.nan

def remove_dot(x):
    try:
        t = re.findall(r'\.\d+', x)
    except:
        return ''
    if t:
        x = x.replace(t[0], '')
        return x
    else:
        return x

df['parse3'] = df['parse3'].apply(remove_dot)
    
df['rooms'] = df['parse3'].apply(lambda x: rooms(x, mode = 'rooms'))
df['area'] = df['parse3'].apply(lambda x: rooms(x, mode = 'area'))
df['land_area'] = df['parse3'].apply(lambda x: rooms(x, mode = 'land_area'))

df.drop('parse3', axis = 1, inplace = True)

In [49]:
# Data after "parse3" column preprocessing
df.head()

,price_usd,offer_type,type,city,region,rooms,area,land_area
0,71 970 $,sell,apartments,Kyiv,Kyiv,1,57,NaN
1,68 000 $,sell,apartments,Kyiv,Kyiv,1,68,NaN
2,53 000 $,sell,house,Vinnytsia,Vinnytska,NaN,134,7
3,36 500 $,sell,apartments,Vinnytsia,Vinnytska,1,36,NaN
4,16 582 $,sell,apartments,Vinnytsia,Vinnytska,1,40,NaN


### Price preprocessing 

In [50]:
df['price_hr'] = df[df['price_usd'].str.contains('грн')]['price_usd']
df.loc[df['price_usd'].str.contains('грн'), 'price_usd'] = 0

In [51]:
df.head()

,price_usd,offer_type,type,city,region,rooms,area,land_area,price_hr
0,71 970 $,sell,apartments,Kyiv,Kyiv,1,57,NaN,NaN
1,68 000 $,sell,apartments,Kyiv,Kyiv,1,68,NaN,NaN
2,53 000 $,sell,house,Vinnytsia,Vinnytska,NaN,134,7,NaN
3,36 500 $,sell,apartments,Vinnytsia,Vinnytska,1,36,NaN,NaN
4,16 582 $,sell,apartments,Vinnytsia,Vinnytska,1,40,NaN,NaN


In [52]:
def muney(x):
    if x:
        x = str(x).replace(' ', '')
        x = re.findall(r'\d+', x)
        if x:
            return int(x[0])
        else:
            return 0
    else:
        return 0

df['price_usd'] = df['price_usd'].apply(muney)
df['price_hr'] = df['price_hr'].apply(muney)

hr_to_usd = 0.041

In [53]:
df['price_usd'] = df['price_usd'].astype('int')
df['price_hr'] = df['price_hr'].astype('int')

df_hr = df[df['price_hr'] > 0]
hr_index = df_hr.index

df.loc[hr_index, 'price_usd'] = df_hr['price_hr'] * hr_to_usd

df.drop('price_hr', axis = 1, inplace = True)

In [54]:
# Data after price preprocessing
df.head()

,price_usd,offer_type,type,city,region,rooms,area,land_area
0,71970.0,sell,apartments,Kyiv,Kyiv,1,57,NaN
1,68000.0,sell,apartments,Kyiv,Kyiv,1,68,NaN
2,53000.0,sell,house,Vinnytsia,Vinnytska,NaN,134,7
3,36500.0,sell,apartments,Vinnytsia,Vinnytska,1,36,NaN
4,16582.0,sell,apartments,Vinnytsia,Vinnytska,1,40,NaN


### Calculating price for square meter and price for 1/100 of hectar 

In [55]:
df[['rooms', 'area', 'land_area']] = df[['rooms', 'area', 'land_area']].astype('float')
df['price_sqm'] = df['price_usd'] / df['area']
df['price_hndr'] = df['price_usd'] / df['land_area']

df[['price_sqm', 'price_hndr']] = df[['price_sqm', 'price_hndr']].round(2)

In [56]:
df.head()

,price_usd,offer_type,type,city,region,rooms,area,land_area,price_sqm,price_hndr
0,71970.0,sell,apartments,Kyiv,Kyiv,1.0,57.0,NaN,1262.63,NaN
1,68000.0,sell,apartments,Kyiv,Kyiv,1.0,68.0,NaN,1000.00,NaN
2,53000.0,sell,house,Vinnytsia,Vinnytska,NaN,134.0,7.0,395.52,7571.43
3,36500.0,sell,apartments,Vinnytsia,Vinnytska,1.0,36.0,NaN,1013.89,NaN
4,16582.0,sell,apartments,Vinnytsia,Vinnytska,1.0,40.0,NaN,414.55,NaN


In [57]:
# Saving preprocessed data to csv file
df.to_csv('ria_ready.csv', index = False)